# Intro to MongoDB

## Set up

This lesson assumes you have already created a free cluster on [MongoDB Atlas](http://bit.ly/MDB_Atlas) and that you have [loaded the sample datasets](http://bit.ly/AtlasSampleData) into that cluster.

### Install PyMongo

PyMongo is a Python distribution containing tools for working with MongoDB, and is the recommended way to work with MongoDB from Python. The installation will take a couple of minutes. You can learn more about PyMongo in the [official documentation](http://bit.ly/cyd-pymongo).

In [ ]:
conda install pymongo

### Install dnspython

[dnspython](http://bit.ly/cyd-dnspython) is a DNS toolkit for Python. You will need dnspython in order to successfully use your Atlas connection string. The installation will take a couple of minutes.

In [ ]:
conda install dnspython

### Restart the kernel

After installing PyMongo and dnspython, you will likely need to restart your kernel.  You can do so in Jupyter Lab by selecting **Kernel > Restart Kernel...** and clicking **Restart**.

### Connect to your demo cluster

Now we are ready to setup a connection to your cluster.  
1. Navigate to [Atlas](http://bit.ly/Atlas_Login).
1. In the **CONTEXT** menu in the upper-left, select your project.
1. Click **Clusters** in the left navigation pane.
1. In the right pane, open your cluster by clicking **Cluster0**.
1. Click the **CONNECT** button on the right side of the page. A **Connect to Cluster 0** dialog appears.
1. In the **Whitelist your connection IP address** section, click **Add Your Current IP Address**. Then click **Add IP Address**. This will whitelist your current IP address, so you can access your cluster.  Note that if your IP Address changes, you will need to add your new IP address.
1. In the **Create a MongoDB User** section, input a new username and password and click **Create MongoDB User**. Be sure to remember these credentials as you will need them again later.
1. Click **Choose a connection method**.
1. Click **Connect Your Application**.
1. In the **Choose your driver version** step, select **Python** as your driver and **3.6 or later** as your version.
1. In the **Connection String Only** section, click **Copy**.
1. Paste the connection string in the code below.
1. Replace `<password>` with a valid password in the connection string you just pasted.
1. Run the code below in order to connect to your database and execute a simple query. If information about a shipwreck is displayed, you are successfully connected.  If no output is displayed, ensure that you have loaded the sample dataset by navigating to the **Collections** pane in your Atlas cluster.  

In [ ]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient("PASTE-YOUR-CONNECTION-STRING-HERE")

db = client.sample_geospatial
shipwrecks = db.shipwrecks
shipwrecks.find_one()

# STOP HERE AND WAIT FOR INSTRUCTIONS

## Lesson: CRUD Operations

In this section, we will work through executing the CRUD operations in Python.  You can also perform the CRUD operations in Atlas (navigate to your cluster and then click the Collections tab) or in Compass.  

### Create

The first thing you'll likely want to do is create data in your database.  We can do this with an [insert_one](http://bit.ly/cyd-insert-one) or [insert_many](http://bit.ly/cyd-insert-many) operation. 

Let's say we want to create data about a new shipwreck.  We can choose to insert as much or as little data about the shipwreck as we'd like.

Note that MongoDB requires every document to contain a unique `_id`.  If you don't include an id in the document, MongoDB will create one for you.  The id will be returned as part of the results of `insert_one`.  

In [ ]:
# Create a Python dictionary for the new document we want to create
new_shipwreck = {
    'feature_type': 'Wrecks - Submerged, dangerous',
    'latdec': 41.878113,
    'londec': -86.929799,
    'name': 'A glorious crash',
    'coordinates': [-86.929799, 41.878113]
}

# Insert our new shipwreck into our collection
results = shipwrecks.insert_one(new_shipwreck)

# Print the new document's _id, which MongoDB automatically created for us
print("The id of the document is " + str(results.inserted_id))

## Read

Now that we have inserted a document, let's read it from the database.  We'll use [find_one](http://bit.ly/cyd-find-one) to retrieve one document from our shipwrecks collection that meets our query critera.  Let's search for the document we just created by using its _id.

In [ ]:
from bson.objectid import ObjectId

shipwrecks.find_one({'_id': ObjectId('PASTE-THE-DOCUMENT-ID-HERE')})

We successfully retrieved the documented we just created from our database!

Now, let's use `find_one` to search for one document in our shipwrecks collection that has the name we used in our `new_shipwreck` dictionary. 

In [ ]:
shipwrecks.find_one({'name': 'A glorious crash'})

Let's say we want to search for all shipwrecks that occurred in the northern hemisphere.  We can use [find](http://bit.ly/cyd-find) to search for all documents in our shipwrecks collection that have a `latdec` greater than 0.  We'll use the `$gt` comparison query operator to search for numbers greater than 0.  See http://bit.ly/cyd-query-comparison for more comparison query operators.

In [ ]:
import pprint

print("Our collection contains " + str(shipwrecks.count_documents({'latdec': {'$gt': 0}})) + " shipwrecks in the northern hemisphere.\n")

# We don't want to print all of those results, so let's print the first 3.
for wreck in shipwrecks.find({'latdec': {'$gt': 0}}).limit(3):
    pprint.pprint(wreck)


You have the power to build complex queries to search for exactly what you need.  The queries above all have a single query parameter, but you can add multiple query parameters to your query if you'd like.

### Update

You may find that you need to make changes to a document after it is in your database.  In this case, we'd use [update_one](http://bit.ly/cyd-update-one).  Let's update the document we inserted earlier.  `update_one` has two mandatory parameters.  First, you indicate the filter (query) that should be used to find the document to update.  Second, you indicate how the document should be updated.

In [ ]:
result = shipwrecks.update_one( {'name': 'A glorious crash'}, {'$set': {'feature_type': 'Wrecks - Visible', 'name': 'A horrible crash'}} )

print(str(result.modified_count) + " document was updated")

# Search for a document with the new name
shipwrecks.find_one({'name': 'A horrible crash'})

We can also update many documents at a time using [update_many](http://bit.ly/cyd-update-many). Let's update documents in the northern hemisphere so they have a new field indicating their hemisphere.

In [ ]:
result = shipwrecks.update_many( {'latdec': {'$gt': 0}}, {'$set': {'hemisphere': 'northern'} } )

print(str(result.modified_count) + " documents were updated.\n")

# Let's print 3 of the updated docs, so we can see the updates
for wreck in shipwrecks.find({'latdec': {'$gt': 0}}).limit(3):
    pprint.pprint(wreck)

The examples above use the `$set` Update Operator to indicate what fields should be set in the docs to be udpated.  See the [official documentation](http://bit.ly/cyd-docs-update) for a list of all Update Operators.

### Delete

You may also need to delete documents.  You can use [delete_one](http://bit.ly/cyd-delete-one) or [delete_many](http://bit.ly/cyd-delete-many).  These functions behave similarly to the ones we have already used, so we'll only show an example of `delete_one` today.  

Let's delete the document with the name 'A horrible crash'.

In [ ]:
result = shipwrecks.delete_one( {'name': 'A horrible crash'} )
print(str(result.deleted_count) + " document was deleted")

## Optional Lesson: MongoDB Charts Demo

### Setup MongoDB Charts

MongoDB Charts is a tool to create visual representations of your MongoDB data. 

#### Open MongoDB Charts
1. Open a new tab in your browser.
1. Navigate to [Atlas](http://bit.ly/cyd-atlas-login).
1. If prompted, authenticate.
1. In the Context menu in the upper-left corner, select your project.
1. In the left menu, click **Charts**. 
1. If prompted, click **Activate MongoDB Charts**.  MongoDB Charts will open.

#### Add a Data Source
A Data Source is a reference to a MongoDB collection that contains the data you wish to visualize.
1. In the top menu, click **Data Sources**.
1. Click **New Data Source**.
1. Select your project and click **Connect**.
1. When prompted to choose a collection, select the checkbox on the **ShipInfo** row and click **Set Permissions**.
1. Click **Publish Data Source**.

#### Create a Dashboard
A dashboard is a place where you can display one or more charts.
1. In the top menu, click **Dashboards**.
1. Click **New Dashboard**.
1. Leave the default title and description for your dashboard and click **Create**.

#### Navigate back to Atlas
1. In the upper-left corner, click **< Atlas** to navigate back to Atlas.


### Try MongoDB Charts

Now you're ready to try MongoDB Charts!

Information about the steps is included in *italics*.

1. Open your browser and find the tab that is displaying Atlas.
1. In the left menu, click **Charts**. *MongoDB Charts is a tool to create visual representations of your MongoDB data. You might use Charts if you want to display data from your apps to your stakeholders.  You might also use Charts to display data inside of your app itself.*
1. Click **My Dashboard**.

### Create a Map of the Shipwrecks
1. Click **Add Chart**.
1. *The first thing we need to do is choose what data we want to visualize. Let's say we want to see a map of shipwrecks where only the distributed remains are still available.  We also want to be able to see at a glance the history of the wreck.*
1. In the **Choose a Data Source** selection box, select **ShipInfo.shipwrecks**. *Now we can see a list of fields I can choose to use in my chart.  Let's begin by creating a visualization of where the shipwrecks occurred.*
1. In the Chart Type selection box, choose **Geospatial** and then click **Scatter**.
1. Drag **coordinates** from the Fields pane and drop it on the **Coordinates** section of the Encode pane. *Now we can see a map of the shipwrecks. This chart type only supports 1,000 documents, so we'll filter some of the results in a moment.*
1. Drag **history** from the Fields pane and drop it on the **Color** section of the Encode pane. *Let's give our users a visual indicator of the wreck feature types.*
1. Click the **Filter** tab.  
1. Drag **feature_type** from the Fields pane and drop it on the **Chart Filters** section of the **Filter** pane.  
1. **Deselect All** of the boxes in the Chart Filters.
1. Select the checkbox beside **distributed remains of wreck**. *Now we can see a nice visualization of the shipwrecks. We have limited our results to only display shipwrecks where the feature type is distributed reamins of wreck.*
1. Above the map, click **Enter a title for your chart**, input **Map of wrecks with distributed remains**, and click the checkbox. *Let's give our map a title.*
1. *Now that we have a nice visualization, let's save it.* In the upper-left corner, click **Save and Close**.
1. *We want our dashboard to look nice, so let's adjust the size of the chart.* Drag the bottom-right corner of the chart to make it bigger.

### Chart the Wreck Types
1. *Let's create a different type of visualization.  Let's say we want to see what wreck types are the most popular.*
1. Click **Add Chart**.
1. In the **Choose a Data Source** selection box, select **ShipInfo.shipwrecks**. *Once again, we need to select our data source. If we had multiple collections, we could select a different data source here.*
1. In the Chart Type selection box, choose **Circular**.
1. Drag **feature_type** from the Fields pane and drop it on the **Label** section of the Encode pane. 
1. Drag **feature_type** from the Fields pane and drop it on the **Arc** section of the Encode pane. *Now we can see a nice visualization of the shipwrecks.*
1. *Let's give our map a title.*  Above the map, hover your mouse over **Enter a title for your chart**, click the Pencil button that appears, input **Types of Wrecks**, and click the Checkmark button. 
1. *Now that we have a nice visualization, let's save it.* In the upper-left corner, click **Save and Close**.
1. Adjust the size of the chart if you'd like.

*Now we have two charts on our dashboard.  We could add several more charts to this dashboard if we'd like.  We could then share this dashboard with project stakeholders so they could easily see what is happening in the app.*

### Embed the Wreck Types Chart

*In addition to sharing the dashboard, you may want to display the chart in your app or on a webpage like a blog.*

1. In your MongoDB Charts dashboard, hover over the Types of Wrecks chart, click the Options button (**...**) in the upper-right corner of the chart, and select **Embed Chart**.
1. *Some data is private and you will only want certain users to be able to view it.  MongoDB Charts has authentication options to help you keep your data secure. In this case, our data is not private, so we'll enable unauthenticated access.* Click **Enable embedding options**.
1. Switch the **Enable external embedding for charts that use this data source** option to **ON**.
1. Choose the **Unauthenticated or Verified Signature** option.
1. Click **Go Back**.
1. Switch **Enable unauthenticated access** to **ON**. 
1. Copy the entire src url from inside the EMBED CODE. You should not copy the quotations. For example, the src url you copy should look something like `https://charts.mongodb.com/charts-team1-ximow/embed/charts?id=69fa82ce-d97f-4dc1-bc9b-404f8af7db6b&tenant=50e4d10e-b0a9-4e28-8c62-02c61b58f69a`. *If you were going to embed this in a webpage, you could copy this entire code snippet and paste it into your webpage. Since we want to embed in a Jupyter Notebook, we're going to do things a little differently.*
1. Inside of this Jupyter notebook, paste the url in the code snippet below and run it.

*In only a few minutes, you've created a nice visualization of data and displayed it externally.  You didn't have to do any programming to make that happen.  Keep this capability in mind as you build your apps.  It might save you a lot of time to use MongoDB Charts instead of manually programming visualizations.*


In [ ]:
from IPython.display import IFrame    
IFrame('PASTE-YOUR-SRC-URL-HERE', width=640, height=480)